In [1]:
# All Imports
from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from amazon.api import AmazonAPI
import io
import re
from amazon_scraper import AmazonScraper
from nltk.corpus import sentiwordnet as swn

%matplotlib inline

In [2]:
# This method is done for you. Make sure to put your credentials in amazon.cfg.
def get_amazon(config_file):
    """ Read the config_file and construct an instance of AmazonAPI.
    Args:
      config_file ... A config file in ConfigParser format with Amazon credentials
    Returns:
      An instance of AmazonAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    amazon = AmazonScraper(
                   config.get('amazon', 'AMAZON_ACCESS_KEY'),
                   config.get('amazon', 'AMAZON_SECRET_KEY'),
                   config.get('amazon', 'AMAZON_ASSOC_TAG'))
                   
    return amazon

amzn = get_amazon('amazon.cfg')
print('Established Amazon connection.')

Established Amazon connection.


In [ ]:
# Fetch the 100 most viewes Product IDs of keyword (i.e 'Microsoft Surface') which falls in category (i.e 'Electronics').

pid_lst = []
for q in itertools.islice(amzn.search(Keywords='Microsoft Surface', SearchIndex='Electronics'), 100):
    print(q.asin)
    pid_lst.append(q.asin)

In [ ]:
# Store all the Product IDs in a file so that we can use it in future. 

fp = open('MS_PID.txt','w')
for item in pid_lst:
    fp.write("%s\n" %item)

fp.close()

In [ ]:
# Reading the IDs from file and put it in list. 

pid = []
pfile = open('MS_PID.txt','r')
for i in pfile.read().split():
    pid.append(i)
    
pfile.close()

In [ ]:
# To check the number of Product IDs we have fetched. 

len(pid)

In [3]:
# Take one by one Product ID and search for it. 

#pd = 'B00MA7A876'
pd = 'B00KDI9C4O'
p = amzn.lookup(ItemId=pd)

In [ ]:
p.url

In [4]:
# Download the reviews from amazon page by page and store the id and review text in list form. 

def Get_Reviews(pid):
    
    rs = amzn.reviews(ItemId=pid)
    myReviews = []
    myRID = []

    for r in rs.full_reviews():
    
        myRID.append(r.id)
        myReviews.append(r.text)
           
        if len(myReviews) >= 100:
            break
    
    return myRID, myReviews

In [43]:
# Calling the Get_Reviews method and get the data. 


myRID, myReviews = Get_Reviews(pd)

In [41]:
len(myRID)

10

In [44]:
myReviews

[u'None',
 u'None',
 u'None',
 u'None',
 u'Great computer and is more productive with touch screen after getting used to windows 8.1. Has many productivity features that I have not learned the full potential of yet.  Only problem is mouse pad has probably 10 features (not sure yet it does a ton of annoying things) that annoy and take up most the RAM, I am sure there is a way to adjust the mouse settings.  Touch screen is incredibly fast, mouse does its own thing and uses up much of the RAM.\nSecondly this computer has a great processor just a little on the downside for RAM.  I have purchased a fast SD CARD for additional $20 and Windows Ready Boost has been helping tremendously for Word and Adobe documents.  Most likely going to put 8GB RAM into it so I can utilize the full potential of the computer.',
 u'awesome my husband loves this!',
 u"Wifi problems galore. At least 10 calls to hp plus I mailed it in twice. They had me change the router, I did. Still the same problem...They said i

In [45]:
# To store the reviews in text file and named as it's Product ID. 

def Review_Store_File(p,myRID,myReviews):
    
    review_file = open('Reviews_Data/' + p.asin + '.txt','a')
    for i in range(len(myRID)):

        review_file.write(myRID[i] + "->" + myReviews[i].replace("\n", " ").encode('utf-8') + "\n")


    review_file.close()

In [46]:
Review_Store_File(p,myRID,myReviews)

In [11]:
# To store the Item into items.txt to keep track of all items.


def Item_Store_File(p):

    items_file = open('items.txt',"a")
    items_file.write(p.asin + "\t" + p.title.encode('utf-8') + "\t" + p.manufacturer + "\n")

    items_file.close()


In [12]:
Item_Store_File(p)

In [13]:
# To see the list of Items stored in 'items.txt' file. Here 3 columns are there. first depicts the Product IDs, 
# second gives Product titles and last column gives Manufacturer details.  

def Items(itm):

    itm_file = open(itm,"r")
    print itm_file.read()
    itm_file.close()


In [14]:
Items('items.txt')

B00THEP15O	Dell Inspiron 15.6 Inch Laptop	Dell Computers
B00ZOO1RFO	Dell Inspiron i3543 15.6-Inch Touchscreen Laptop	Dell Computers
B015JVB9C2	Dell Inspiron i3000-5099SLV Touchscreen Laptop	Dell Marketing USA, LP
B010DOD2G6	Dell Inspiron 15 5000 15.6 Inch Laptop	Dell Marketing USA, LP
B00VKL0I96	Dell Latitude E6420 Premium 14.1-Inch Business Laptop	Dell Computers
B010DODAWM	Dell Inspiron 11 3000 Series 2-in-1  11.6 Inch Laptop	Dell Marketing USA, LP
B013PK8LGQ	Dell Inspiron 3000 15.6-Inches Windows 8.1 Laptop 	Dell Computers
B012BPRLO4	Dell Latitude E6400 Laptop Core 2 Duo	Dell Computers
B00YD1OQ1G	Dell Inspiron N2840 14-Inch Laptop	Dell Computers
B014B9IS1S	2015 Newest Dell Chromebook, 11.6 inch HD Non-Touch LCD	Dell Computers
B00V2KX5JA	2015 Newest Model Dell Inspiron 15 Laptop Computer	Dell Computers
B00ZS64K7K	Dell Inspiron 15 i5548-4167SLV - 15.6" Touchscreen	Dell Computers
B00KMRGF28	Dell Inspiron i3531-1200BK 16-Inch Laptop,Black	Dell Computer
B00Z1R85O0	2015 Newest Dell Inspiro